# Proyecto 7: Deploy de sistema de recomendación con Watson
En este proyecto llevaremos a cabo la puesta en producción del modelo entrenado en el proyecto 5. Es decir, lo subirmos la nube de IBM y utilizando llamados a la API de Watson tendremos acceso a él para realizar predicciones.

In [1]:
import joblib
import warnings
import numpy as np
from pathlib import Path

from sklearn.datasets import load_files
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

warnings.filterwarnings("ignore")

## IBM Watson

**1) Cargá** la biblioteca `WatsonMachineLearningAPIClient`

In [2]:
from ibm_watson_machine_learning import APIClient

**2) Creá** variable con las credenciales que necesita `Watson`. Ellas son: `url, access_key, username, password e instance_id`
> **Nota:**
> La versión V2 de la API de Machine Learning de Watson no requiere ciertas variables como `access_key, username, password e instance_id`

In [3]:
api_key = 'e0jsalFhF4CKCyol1bvMQ1LemqjgTjqY9L8U49jv_1rX'
space_id = 'c73391d4-f6ea-4c7a-b462-57800b84f19f'
url = 'https://us-south.ml.cloud.ibm.com'

wml_credentials = {
    "apikey": api_key,
    "url": url
}

**3) Declará** la variable `client` y guardá en ella al objeto `WatsonMachineLearningAPIClient` con las credenciales como parámetro

In [4]:
client = APIClient(wml_credentials)

client.set.default_space(space_id)

'SUCCESS'

**4) Creá** una variable que guarde las propiedades del modelo. Datos del autor y nombre del proyecto.

In [5]:
sofware_spec_uid = client.software_specifications.get_id_by_name("default_py3.7")

metadata = {
    client.repository.ModelMetaNames.NAME: 'Scikit model',
    client.repository.ModelMetaNames.TYPE: 'scikit-learn_0.23',
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID : sofware_spec_uid
}

**5) Hacé** un pipeline que contenga como primer paso a un `TfidfVectorizer` y como segundo paso, al mejor modelo que hayas obtenido en el proyecto 5. **Entrená** con este pipeline.

In [6]:
moviedir = Path('./dataset/movie_reviews')
movie_reviews = load_files(moviedir, shuffle=True)

X_train, X_test, y_train, y_test = train_test_split(movie_reviews.data, movie_reviews.target, test_size = 0.20, stratify=movie_reviews.target, random_state = 12)

eclf = joblib.load(Path('./model/sentiment.pkl'))

vectorizer = TfidfVectorizer()
pipeline = make_pipeline(vectorizer, eclf)

pipeline.fit(X_train, y_train)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('linearsvc', LinearSVC(C=1, random_state=0))])

**6) Subí** al modelo a IBM Cloud usando `client.repository.store_model` con los parámetros correctos.

In [7]:
publish_model = client.repository.store_model(model=pipeline, meta_props=metadata, training_data=X_train, training_target=y_train)

**7) Obtené** el `uid` del modelo y guardalo en una variable.

In [8]:
published_model_uid = client.repository.get_model_uid(publish_model)
models_details = client.repository.list_models()

------------------------------------  ------------  ------------------------  -----------------
ID                                    NAME          CREATED                   TYPE
487bf8ed-2ddd-4732-a05a-af5e99cdf983  Scikit model  2020-12-27T00:23:58.002Z  scikit-learn_0.23
be8fc4f5-7a08-4994-b14f-c7e2de68f955  Scikit model  2020-12-27T00:19:13.002Z  scikit-learn_0.23
6d1fc835-069d-4e3c-820c-9a133fc05c91  Scikit model  2020-12-27T00:16:21.002Z  scikit-learn_0.23
d18def2a-4c38-4a2c-8e8d-3090619336f3  Scikit model  2020-12-27T00:12:17.002Z  scikit-learn_0.23
------------------------------------  ------------  ------------------------  -----------------


**8) Cargá** el modelo basado en su `uid` y utilizalo para realizar la predicción sobre el conjunto de test

In [9]:
loaded_model = client.repository.load(published_model_uid)
test_predictions = loaded_model.predict(X_test)

**9) Mostrar** el `classification_report` obtenido

In [10]:
print(classification_report(y_test, test_predictions))

              precision    recall  f1-score   support

           0       0.87      0.82      0.85       200
           1       0.83      0.88      0.85       200

    accuracy                           0.85       400
   macro avg       0.85      0.85      0.85       400
weighted avg       0.85      0.85      0.85       400

